# New Data Analysis

In [1]:
dataDir='Data/'

In [2]:
import json
import pandas as pd
import re

## Openning the data¶

In [3]:
#function that open the file and create a "good" dataset
def createDataFrame(file):
    ##lists of 50 data are loaded and then merged
    #create list of dataframes and open file
    dataframes=[]
    file=open(dataDir+file, 'r')
    
    #counter is important for the index
    c=0
    while True:
        #each line is 50 data (as limit=50 when downloading)
        txt=file.readline()
        #possibly stop
        if txt=='':
            break

        #load the data with right index
        j=json.loads(txt)
        l=len(j)
        dataframes.append(pd.DataFrame(j, index=list(range(c, c+l))))
        
        c+=l
    #merge all datasets and return result
    return pd.concat(dataframes)

In [4]:
dfEvent=createDataFrame('event.json')

In [5]:
dfEvent.columns

Index(['beneficiaries_id', 'canceled', 'checks_payable_to_address',
       'contributions_info', 'distribution_paid_for_by', 'end_date',
       'end_time', 'entertainment', 'hosts_id', 'id', 'is_presidential',
       'make_checks_payable_to', 'more_details', 'notes', 'other_members',
       'postponed', 'resource_uri', 'rsvp_info', 'start_date', 'start_time',
       'venue_id'],
      dtype='object')

In [6]:
#dfEvent

--
### Number of events:

In [7]:
#number of tracked events:
len(dfEvent)

24108

--
### Contributions for each event:

In [8]:
##quick look at what it looks like:
#dfEvent['contributions_info']

#### looking for host contribution:

see https://regex101.com/r/pj3UC4/1 for more info on the reg expr

In [9]:
def getHostContrib(s):
    reg=r'((h|H)ost[ Commitee\:]+\$?[,0-9]+|\$?[,0-9]+[ per\-PACto]+(h|H)ost( [^$]|;|\n|$)|(h|H)ost - \$[,0-9]+)'
    reg_expr=re.compile(reg)
    reg_expr2=re.compile(r"(h|H)ost")
    
    #determinating if there is a host contribution info:
    hosts = re.finditer(reg_expr2, s)
    h=False
    for hostNum, host in enumerate(hosts):
        h=True
        
    
    matches = re.finditer(reg_expr, s)
    c=0
    l=[]
    for matchNum, match in enumerate(matches):
        l.append(match.group())
        c+=1
    #c is the number of hosts info caught

    #if not found, but there is some host info
    if c==0 and h:
        print(s)
        
        
        

#### Type of contribution_info string that gives me toubles:

In [10]:
dfEvent['contributions_info'].apply(getHostContrib)

Co-Host - Raise: $5,000 / Co-Host - Give: $2,500 / Sponsor: $1,000 / Guest: $100
$5,000 per person $100,000 per couple to Co-Host
Host Levels: Platinum $1,000 Gold $500 Silver $250 Suggested $50
Host Levels: Platinum $1,000 Gold $500 Silver $250 Suggested $50
Host Levels: Platinum $1,000 Gold $500 Silver $250
Hosts: Platinum $1,000 Gold $500 Silver $250 Bronze $100 Suggested 50
Day One Marco Couple $10,800 (Event co-Host) Day One Marco Supporter $5,400 (Photo and Dinner) General Attendee $500 (Reception)
$5,000 Individual Host | $2,500 Individual Sponsor | $1,000 Individual Guest 
$25,000 per person $100,000 per couple and spot on host committee
Event Chairs $5400 Event Vice Chairs $2700 Platinum Hosts $1000 Gold Hosts $500 Silver Hosts $250 Guests $100 Per Person
Event Chairs $5,400 Event Vice Chairs $2,700 Platinum Hosts $1,000 Gold Hosts $500 Silver Hosts $250 Guests $100 Per Person
$2500 Event Chairs $1000 Event Vice Chairs $500 Platinum Hosts $250 Gold Hosts $100 Guests
Hosts: $25

TypeError: expected string or bytes-like object

In [11]:
f=open('pbs', 'r')
l=f.readlines()
f.close()
pbs=len(l)
del l
print('there are', pbs, 'problems over', len(dfEvent), 'cases, ie:', 100*pbs/len(dfEvent), '% of errors')

there are 418 problems over 24108 cases, ie: 1.7338642774182844 % of errors


I think that this is acceptable, isn't it?

# Open secret data

In [12]:
dfContrib=createDataFrame("candContrib.json")
dfContrib.columns

Index(['cid', 'indivs', 'org_name', 'pacs', 'total', 'year'], dtype='object')